In [2]:
pip install ultralytics opencv-python pandas numpy matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.8/898.8 kB 16.8 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import shutil
from sklearn.model_selection import train_test_split
import yaml

In [ ]:
dataset_path = '/kaggle/input/indian-currency-notes/NOTES'
output_path = '/kaggle/working/dataset'
train_dir = os.path.join(output_path, 'train')
val_dir = os.path.join(output_path, 'val')
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

original_class_names = [
    'INDIA100NEW', 'INDIA100OLD', 'INDIA10NEW', 'INDIA10OLD',
    'INDIA20', 'INDIA200', 'INDIA500', 'INDIA50NEW', 'INDIA50OLD'
]
modified_class_names = [name.replace('INDIA', '') for name in original_class_names]

class_name_to_id = {original: idx for idx, original in enumerate(original_class_names)}

for class_name in os.listdir(dataset_path):
    class_path = os.path.join(dataset_path, class_name)
    if os.path.isdir(class_path):
        class_id = class_name_to_id[class_name]

        images = [os.path.join(class_path, img) for img in os.listdir(class_path) if img.endswith(('.jpg', '.png', '.jpeg'))]

        train_images, val_images = train_test_split(images, test_size=0.2, random_state=42)

        modified_class_name = class_name.replace('INDIA', '')  # Remove "INDIA" from class name
        train_class_dir = os.path.join(train_dir, modified_class_name)
        val_class_dir = os.path.join(val_dir, modified_class_name)
        os.makedirs(train_class_dir, exist_ok=True)
        os.makedirs(val_class_dir, exist_ok=True)

        for img_path in train_images:
            img_name = os.path.basename(img_path)
            label_path = os.path.join(train_class_dir, img_name.replace('.jpg', '.txt').replace('.png', '.txt').replace('.jpeg', '.txt'))
            
            shutil.copy(img_path, train_class_dir)
            
            with open(label_path, 'w') as label_file:
                label_file.write(f"{class_id} 0.5 0.5 1.0 1.0\n")  # YOLO format: class_id x_center y_center width height

        for img_path in val_images:
            img_name = os.path.basename(img_path)
            label_path = os.path.join(val_class_dir, img_name.replace('.jpg', '.txt').replace('.png', '.txt').replace('.jpeg', '.txt'))
            
            shutil.copy(img_path, val_class_dir)
            
            with open(label_path, 'w') as label_file:
                label_file.write(f"{class_id} 0.5 0.5 1.0 1.0\n")

print("Dataset prepared successfully!")

data_yaml = {
    'train': '/kaggle/working/dataset/train',
    'val': '/kaggle/working/dataset/val',
    'nc': len(modified_class_names),
    'names': modified_class_names
}

yaml_file_path = '/kaggle/working/dataset/data.yaml'
with open(yaml_file_path, 'w') as yaml_file:
    yaml.dump(data_yaml, yaml_file, default_flow_style=False)

print(f"data.yaml file saved at: {yaml_file_path}")


Dataset prepared successfully!
data.yaml file saved at: /kaggle/working/dataset/data.yaml


In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')  
model.train(data='/kaggle/working/dataset/data.yaml', epochs=50, imgsz=640, batch=16)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 79.1MB/s]


Ultralytics 8.3.47 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/kaggle/working/dataset/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxe

100%|██████████| 755k/755k [00:00<00:00, 17.2MB/s]
2024-12-07 13:48:38,547	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-12-07 13:48:39,547	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Overriding model.yaml nc=80 with nc=9

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

...


50 epochs completed in 0.278 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.3MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.47 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 168 layers, 3,007,403 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:03<00:00,  3.75it/s]


                   all        360        360      0.999          1      0.995      0.995
                100NEW         40         40      0.999          1      0.995      0.995
                100OLD         40         40      0.999          1      0.995      0.995
                 10NEW         40         40      0.999          1      0.995      0.995
                 10OLD         40         40      0.999          1      0.995      0.995
                    20         40         40      0.999          1      0.995      0.995
                   200         40         40      0.999          1      0.995      0.995
                   500         40         40      0.999          1      0.995      0.995
                 50NEW         40         40      0.999          1      0.995      0.995
                 50OLD         40         40      0.999          1      0.995      0.995
Speed: 0.2ms preprocess, 2.1ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to runs/detect

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6, 7, 8])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x78f4dc54cfa0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.0470

/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [ ]:
model.export(format='torchscript') 

Ultralytics 8.3.47 🚀 Python-3.10.14 torch-2.4.0 CPU (Intel Xeon 2.00GHz)
Model summary (fused): 168 layers, 3,007,403 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from 'runs/detect/train/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 13, 8400) (6.0 MB)

TorchScript: starting export with torch 2.4.0...
TorchScript: export success ✅ 2.7s, saved as 'runs/detect/train/weights/best.torchscript' (11.9 MB)

Export complete (4.6s)
Results saved to /kaggle/working/runs/detect/train/weights
Predict:         yolo predict task=detect model=runs/detect/train/weights/best.torchscript imgsz=640  
Validate:        yolo val task=detect model=runs/detect/train/weights/best.torchscript imgsz=640 data=/kaggle/working/dataset/data.yaml  
Visualize:       https://netron.app


'runs/detect/train/weights/best.torchscript'

In [7]:
!zip -r yolov8n.zip runs/detect/train

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


  adding: runs/detect/train/ (stored 0%)
  adding: runs/detect/train/confusion_matrix_normalized.png (deflated 27%)
  adding: runs/detect/train/results.csv (deflated 64%)
  adding: runs/detect/train/events.out.tfevents.1733579332.90d4dd83e51e.30.0 (deflated 90%)
  adding: runs/detect/train/val_batch1_labels.jpg (deflated 7%)
  adding: runs/detect/train/train_batch0.jpg (deflated 2%)
  adding: runs/detect/train/train_batch3602.jpg (deflated 5%)
  adding: runs/detect/train/PR_curve.png (deflated 25%)
  adding: runs/detect/train/val_batch2_pred.jpg (deflated 5%)
  adding: runs/detect/train/train_batch3601.jpg (deflated 4%)
  adding: runs/detect/train/P_curve.png (deflated 23%)
  adding: runs/detect/train/results.png (deflated 8%)
  adding: runs/detect/train/weights/ (stored 0%)
  adding: runs/detect/train/weights/best.torchscript (deflated 14%)
  adding: runs/detect/train/weights/best.pt (deflated 9%)
  adding: runs/detect/train/weights/last.pt (deflated 9%)
  adding: runs/detect/train/co

In [ ]:
from ultralytics import YOLO
import cv2

model = YOLO('runs/detect/train/weights/best.torchscript')  

input_video_path = '/kaggle/input/moneydata/falling-indian-currency.mp4'  
output_video_path = '/kaggle/working/video.mp4'  

cap = cv2.VideoCapture(input_video_path)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)

    annotated_frame = results[0].plot()

    out.write(annotated_frame)

cap.release()
out.release()

print(f"Processed video saved at: {output_video_path}")


Loading runs/detect/train/weights/best.torchscript for TorchScript inference...

0: 640x640 1 10OLD, 7.4ms
Speed: 2.6ms preprocess, 7.4ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 10OLD, 7.5ms
Speed: 5.3ms preprocess, 7.5ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 10OLD, 7.4ms
Speed: 2.8ms preprocess, 7.4ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 10OLD, 7.4ms
Speed: 2.7ms preprocess, 7.4ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 10OLD, 7.5ms
Speed: 3.7ms preprocess, 7.5ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 10OLD, 7.5ms
Speed: 3.6ms preprocess, 7.5ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 10OLD, 7.4ms
Speed: 3.0ms preprocess, 7.4ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 10OLD, 7.5ms
Speed: 3.6ms preprocess, 7.5ms i

In [ ]:


input_video_path = '/kaggle/input/moneydata/falling-old-indian-currency-notes-bank-money-SBV-338739268-preview.mp4' 
output_video_path = '/kaggle/working/video1.mp4'  

cap = cv2.VideoCapture(input_video_path)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)

    annotated_frame = results[0].plot()

    out.write(annotated_frame)

cap.release()
out.release()

print(f"Processed video saved at: {output_video_path}")
